In [1]:
# -----------------------------------------------------------------------
# Script para clasificación de la mediana y TOP 3 por zonas.
#
# Autor: Rodrigo Hernández A.
# Fecha: 2 de Octubre, 2023.
# Tutora: Dra. Nery Sofía Huerta.
#
# Descripción:
# Este script realiza la clasificación de la mediana para comunas por su zona y
# obtener el top 3 de las comunas con más ingresos de registros (por zonas).
#
# Código desarrollado en lenguaje R en el entorno R de Google Colab.
# Proyecto: TFM - Máster en Big Data y Ciencia de Datos - Universidad Internacional de
# Valencia (VIU)
# -----------------------------------------------------------------------
#
#Distribuciones de regiones por cada una de las zonas
#
# Zona 1 (Norte): XV, I, II, III, IV: 24 comunas con juzgados
#     DE ARICA Y PARINACOTA(1 comuna), DE TARAPACA(3 comunas), DE ANTOFAGASTA(6 comunas),
#     DE ATACAMA(6 comunas), DE COQUIMBO(8 comunas)
# Zona 2: V, RM (Centro Metropolitana): 24 comunas
#     DE VALPARAISO(16 comunas), METROPOLITANA DE SANTIAGO(8 comunas))
# Zona 3: VI, VII, VII, XVI, VIII (Centro Sur): 47 comunas
#     DEL LIBERTADOR B. O'HIGGINS(10 comunas), DEL MAULE(11 comunas), DE ÑUBLE(6 comunas),
#     DEL BIOBIO(20 comunas)
# Zona 4: IX, XIV, X (Sur): 35 comunas
#     DE LA ARAUCANIA(15 comunas), DE LOS RIOS(7 comunas), DE LOS LAGOS(13 comunas)
# Zona 5 (Austral): XI y XII: 9
#     DE AISEN DEL GRAL. C. IBAÑEZ DEL CAMPO(5 comunas), DE MAGALLANES Y DE LA ANTARTICA
#     CHILENA(4 comunas)
#---------------------------------------------------------------------

In [2]:
# -- 1 -- Instalación de paquetes necesarios ----
install.packages("tidyverse")
install.packages("writexl")
install.packages("dplyr")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [3]:
# -- 2 -- Importación de paquetes necesarios ----
library(readxl)
library(tidyverse)
library(writexl)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [4]:
# --- Carga y preparación de datos ---
#Fuente de datos: Proviene de la exportación del script proviene del anterior:
#    https://github.com/RodrigoEduard/Codigos-TFM-VIU-2023/blob/Procesamiento_y_Limpieza_de_Datos/TFM_VIU_2023_11_Procesamiento_y_Limpieza.ipynb
#Descarga alternativa:
#    https://docs.google.com/spreadsheets/d/1zHnctwCKipKhKd9b8bucCGNlwfqfMalI
zonas <- read_excel("/content/conduccion_130823_2015_2023.xlsx")
zonas$`MES INGRESO` <- as.numeric(zonas$`MES INGRESO`)
# Nueva columna 'FECHA' combinando 'AÑO INGRESO' y 'MES INGRESO' en el formato "Mmm.AA". Ejwmplo: Enero de 2015: Ene.15
zonas$FECHA <- paste('a', substr(zonas$`AÑO INGRESO`, 3, 4), '.', sprintf("%02d", zonas$`MES INGRESO`), sep = "")
zonas$FECHA <- paste(zonas$`AÑO INGRESO`, sprintf("%02d", zonas$`MES INGRESO`), sep = ".")

In [5]:
# Zona 1 (Norte): XV, I, II, III, IV: 24 comunas con juzgados
# Filtro y tabulación de datos
regiones <- c('DE ARICA Y PARINACOTA', 'DE TARAPACA', 'DE ANTOFAGASTA','DE ATACAMA', 'DE COQUIMBO')
dat <- subset(zonas, `NOMBRE REGION` %in% regiones)
dat = table(dat$`NOMBRE COMUNA`,dat$`FECHA`)

In [6]:
# Calculo de frecuencias relativas
pde_f = 100*(dat/rowSums(dat))
pde_f = as.data.frame(pde_f)

In [7]:
# Clasificar frecuencias en relación a la mediana
max = max(pde_f[,3],na.rm=T)
max = round(max,digits = 0)
mediana = median(pde_f[,3],na.rm=T)
pde_f$clase = 0
pde_f$clase[pde_f$Freq == mediana] <- 'igual'
pde_f$clase[pde_f$Freq > mediana] <- 'sobre'
pde_f$clase[pde_f$Freq < mediana] <- 'bajo'

In [8]:
# Reorganizar y pivotar el dataframe
ZNO1 <- pde_f %>%
  rename(Comuna = Var1) %>%
  group_by(Comuna, Var2) %>%
  summarise(clase = first(clase), .groups = "drop") %>%
  pivot_wider(names_from = Var2, values_from = clase)

In [9]:
# Ajuste de nombres de fechas
nombres_columnas <- colnames(ZNO1)[-1]  # Se toma los nombres de las columnas excepto "Comuna"
partes <- strsplit(nombres_columnas, "\\.") # Separación en años y meses usando el delimitador "."
años <- sapply(partes, `[`, 1)
meses <- sapply(partes, `[`, 2)
meses_abreviados <- months(as.Date(paste0(años, "-", meses, "-01")), abbreviate = TRUE) # Convertir los números de meses a abreviaturas de meses
# Combinando las abreviaturas de los meses con los años
nuevos_nombres_columnas <- c("Comuna", paste0(toupper(substr(meses_abreviados, 1, 1)), tolower(substr(meses_abreviados, 2, 3)), ".", substr(años, 3, 4)))
colnames(ZNO1) <- nuevos_nombres_columnas # Asignando los nuevos nombres de columnas a ZNO1

ZNO1 <- ZNO1 %>%
  select(Comuna, one_of(nuevos_nombres_columnas))

In [ ]:
# Ajuste de nombres de comunas
# (Para comunas de Alto Hospicio y Mejillones que no existían antes de Noviembre 2016)
comunas_ajustar <- c("ALTO HOSPICIO", "MEJILLONES")
meses_ajustar <- c(paste0(toupper(substr(months(as.Date(paste0("20", rep(15, times = 12), "-", sprintf("%02d", 1:12), "-01")), abbreviate = TRUE), 1, 1)),
                          tolower(substr(months(as.Date(paste0("20", rep(15, times = 12), "-", sprintf("%02d", 1:12), "-01")), abbreviate = TRUE), 2, 3)), ".15"),
                   paste0(toupper(substr(months(as.Date(paste0("20", rep(16, times = 10), "-", sprintf("%02d", 1:10), "-01")), abbreviate = TRUE), 1, 1)),
                          tolower(substr(months(as.Date(paste0("20", rep(16, times = 10), "-", sprintf("%02d", 1:10), "-01")), abbreviate = TRUE), 2, 3)), ".16"))

ZNO1 <- as.data.frame(ZNO1)
for (comuna in comunas_ajustar) {
  for (mes in meses_ajustar) {
    ZNO1[ZNO1$Comuna == comuna, mes] <- "sin_juzgado"
  }
}

In [10]:
#Obtención del Top 3 de las comunas con más registros
pde_suma <- as.data.frame(rowSums(dat))
pde_suma$NombreDeLaColumna <- rownames(pde_suma)
rownames(pde_suma) <- NULL
pde_suma <- pde_suma[, c("NombreDeLaColumna", "rowSums(dat)")]
colnames(pde_suma) <- c("Comuna", "Frecuencia")

pde_suma_ordenado <- pde_suma[order(-pde_suma$Frecuencia), ]
top_3_comunas <- pde_suma_ordenado[1:3, ]
print(top_3_comunas)

        Comuna Frecuencia
4        ARICA       9353
14     IQUIQUE       9132
3  ANTOFAGASTA       7774


In [11]:
# Exportación del dataframe ZNO1 en un archivo Excel
library(writexl)
write_xlsx(ZNO1,"01-zonaNorte.xlsx")

In [ ]:
# ZONA 2: V, RM (CENTRO-METROPOLITANA): 24 comunas
#   DE VALPARAISO(16), METROPOLITANA DE SANTIAGO(8))
regiones <- c('DE VALPARAISO','METROPOLITANA DE SANTIAGO')
dat <- subset(zonas, `NOMBRE REGION` %in% regiones)
dat = table(dat$`NOMBRE COMUNA`,dat$`FECHA`)

In [ ]:
# Calculo de frecuencias relativas
pde_f = 100*(dat/rowSums(dat))
pde_f = as.data.frame(pde_f)

In [ ]:
# Clasificar frecuencias en relación a la mediana
max = max(pde_f[,3],na.rm=T)
max = round(max,digits = 0)
mediana = median(pde_f[,3],na.rm=T)
pde_f$clase = 0
pde_f$clase[pde_f$Freq == mediana] <- 'igual'
pde_f$clase[pde_f$Freq > mediana] <- 'sobre'
pde_f$clase[pde_f$Freq < mediana] <- 'bajo'

In [ ]:
# Reorganizar y pivotar el dataframe
ZCM2 <- pde_f %>%
  rename(Comuna = Var1) %>%
  group_by(Comuna, Var2) %>%
  summarise(clase = first(clase), .groups = "drop") %>%
  pivot_wider(names_from = Var2, values_from = clase)

In [ ]:
# Ajuste de nombres de fechas
nombres_columnas <- colnames(ZCM2)[-1]  # Se toma los nombres de las columnas excepto "Comuna"
partes <- strsplit(nombres_columnas, "\\.") # Separación en años y meses usando el delimitador "."
años <- sapply(partes, `[`, 1)
meses <- sapply(partes, `[`, 2)
meses_abreviados <- months(as.Date(paste0(años, "-", meses, "-01")), abbreviate = TRUE) # Convertir los números de meses a abreviaturas de meses
nuevos_nombres_columnas <- c("Comuna", paste0(toupper(substr(meses_abreviados, 1, 1)), tolower(substr(meses_abreviados, 2, 3)), ".", substr(años, 3, 4)))
colnames(ZCM2) <- nuevos_nombres_columnas # Asignando los nuevos nombres de columnas a ZCM2

ZCM2 <- ZCM2 %>%
  select(Comuna, one_of(nuevos_nombres_columnas))

In [ ]:
#Obtención del Top 3 de las comunas con más registros
pde_suma <- as.data.frame(rowSums(dat))
pde_suma$NombreDeLaColumna <- rownames(pde_suma)
rownames(pde_suma) <- NULL
pde_suma <- pde_suma[, c("NombreDeLaColumna", "rowSums(dat)")]
colnames(pde_suma) <- c("Comuna", "Frecuencia")

pde_suma_ordenado <- pde_suma[order(-pde_suma$Frecuencia), ]
top_3_comunas <- pde_suma_ordenado[1:3, ]
print(top_3_comunas)

In [ ]:
# Exportación del dataframe ZNO1 en un archivo Excel
library(writexl)
write_xlsx(ZCM2,"02-zonaCentroMetropolitana.xlsx")